# Exercise 1

In [7]:
import copy

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Tensorboard for visualizing
from torch.utils.tensorboard import SummaryWriter

### Import CIFAR-10

In [8]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root="../CIFAR10/", 
                                        train=True, 
                                        download=True, 
                                        transform=transform)
train_len = int(len(trainset) * 0.8)
val_len = len(trainset) - train_len
trainset, validationset = torch.utils.data.random_split(trainset, [train_len, val_len])
trainloader = torch.utils.data.DataLoader(trainset, 
                                          batch_size=4, 
                                          shuffle=True)
validationloader = torch.utils.data.DataLoader(validationset,
                                          batch_size=1,
                                          shuffle=True)
testset = torchvision.datasets.CIFAR10(root='../CIFAR10/', 
                                       train=False,
                                       download=True, 
                                       transform=transform)
testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=1,
                                         shuffle=False)



Files already downloaded and verified
Files already downloaded and verified


### CNN

In [9]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 84)
        #self.fc2 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.leaky_relu(self.fc1(x))
        #x = F.leaky_relu(self.fc2(x))
        x = self.fc2(x)

        return x

net = Net()

# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.SGD(net.parameters(), lr=0.0001)

writer = SummaryWriter()



### Training

In [10]:
best_accuracy = 0
best_net = 0
for epoch in range(15):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        if(i % 100 == 99):
            print(
                f'\rEpoch {epoch+1} [{i+1}/{len(trainloader)}] - Loss: {loss}',
                end=''
            )
    writer.add_scalar("Loss/train", loss, epoch)
    
    correct = 0
    total = 0
    for i, data in enumerate(validationloader, 0):
        inputs, labels = data
        labels = labels

        outputs = net(inputs)
        pred = torch.argmax(outputs)

        if pred.numpy() == labels[0].numpy():
            correct += 1
        total += 1
    writer.add_scalar("Validation/train", correct/total, epoch)
    print(", accuracy: ", correct/total)
    if correct / total > best_accuracy:
        best_accuracy = correct / total
        best_net = copy.deepcopy(net)
        print(" (new best)")

print('\nFinished Training')
writer.flush()

Epoch 1 [10000/10000] - Loss: 2.2865567207336426, accuracy:  0.0912
 (new best)
Epoch 2 [10000/10000] - Loss: 2.2717552185058594, accuracy:  0.1627
 (new best)
Epoch 3 [10000/10000] - Loss: 2.2479076385498047, accuracy:  0.1759
 (new best)
Epoch 4 [10000/10000] - Loss: 2.122739315032959, accuracy:  0.1815
 (new best)
Epoch 5 [10000/10000] - Loss: 2.3782994747161865, accuracy:  0.221
 (new best)
Epoch 6 [10000/10000] - Loss: 1.7953717708587646, accuracy:  0.2549
 (new best)
Epoch 7 [10000/10000] - Loss: 1.9492322206497192, accuracy:  0.2772
 (new best)
Epoch 8 [10000/10000] - Loss: 2.1912784576416016, accuracy:  0.2984
 (new best)
Epoch 9 [10000/10000] - Loss: 2.1148436069488525, accuracy:  0.315
 (new best)
Epoch 10 [10000/10000] - Loss: 1.537197232246399, accuracy:  0.3246
 (new best)
Epoch 11 [10000/10000] - Loss: 1.5888915061950684, accuracy:  0.3409
 (new best)
Epoch 12 [10000/10000] - Loss: 2.117799758911133, accuracy:  0.3551
 (new best)
Epoch 13 [10000/10000] - Loss: 2.085055589

![Picture missing](1a.png "Validation accuracy each epoch")

### Test

In [11]:
correct = 0
total = 0
for i, data in enumerate(testloader, 0):
    inputs, labels = data

    outputs = best_net(inputs)
    pred = torch.argmax(outputs)

    if pred.numpy() == labels[0].numpy():
        correct += 1
    total += 1

print(correct/total)

0.4017
